In [2]:
import os
import pandas as pd

import warnings

warnings.filterwarnings(action='ignore')

DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/data'
VAL_TO_IDX_DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Server/data'

'''

- target 에 없는 문제 제거 (== 0번 이하 푼 문제)
- 루비 이상 문제 제거
- 언랭크드 문제 제거
- 한국어가 아닌 문제 제거

'''

'\n\n- target 에 없는 문제 제거 (== 0번 이하 푼 문제)\n- 루비 이상 문제 제거\n- 언랭크드 문제 제거\n- 한국어가 아닌 문제 제거\n\n'

In [3]:
problem_df = pd.read_csv(os.path.join(DATA_PATH, 'problem.csv'))
problem_df['problem_id'] = problem_df['problem_id'].astype(int).astype(str)

score_df = pd.read_csv(os.path.join(DATA_PATH, 'score.csv'))
score_df = score_df[~(score_df['problem_id'].isna())].reset_index(drop = True)
score_df = score_df[~(score_df['user_name'].isna())].reset_index(drop = True)
score_df['problem_id'] = score_df['problem_id'].astype(int).astype(str)
score_df['target'] = score_df['target'].astype(int).astype(str)

In [4]:
len(problem_df)

23090

In [5]:
# TODO: target 에 없는 문제 제거 (== 0번 이하 푼 문제)

in_target_problem_df = problem_df.set_index('problem_id').loc[score_df['target'].unique().tolist(), :].reset_index(drop = False)

In [6]:
len(in_target_problem_df) == score_df['target'].nunique()

True

In [7]:
len(problem_df) - len(in_target_problem_df)

3326

In [8]:
len(in_target_problem_df)

19764

In [9]:
# TODO: 루비 이상 문제 제거

'''
{
0	:"Unrated",
1	:"Bronze V",
2	:"Bronze IV",
3	:"Bronze III",
4	:"Bronze II",
5	:"Bronze I",
6	:"Silver V",
7	:"Silver IV",
8	:"Silver III",
9	:"Silver II",
10	:"Silver I",
11	:"Gold V",
12	:"Gold IV",
13	:"Gold III",
14	:"Gold II",
15	:"Gold I",
16	:"Platinum V",
17	:"Platinum IV",
18	:"Platinum III",
19	:"Platinum II",
20	:"Platinum I",
21	:"Diamond V",
22	:"Diamond IV",
23	:"Diamond III",
24	:"Diamond II",
25	:"Diamond I",
26	:"Ruby V",
27	:"Ruby IV",
28	:"Ruby III",
29	:"Ruby II",
30	:"Ruby I",
}

'''

non_ruby_problem_df = in_target_problem_df[in_target_problem_df['level'] <= 25].reset_index(drop = True)

In [10]:
len(in_target_problem_df) - len(non_ruby_problem_df)

419

In [11]:
len(non_ruby_problem_df)

19345

In [12]:
# TODO: 언랭크드 문제 제거

non_unrated_problem_df = non_ruby_problem_df[non_ruby_problem_df['level'] != 0].reset_index(drop = True)

In [13]:
len(non_ruby_problem_df) - len(non_unrated_problem_df)

4687

In [14]:
len(non_unrated_problem_df)

14658

In [22]:
# TODO: 한국어가 아닌 문제 제거
import re

def serch_korean(x):
    if re.search(r"[가-힣]", x): return True
    else: return False

korean_problem_df = non_unrated_problem_df[non_unrated_problem_df['problem_description'].apply(lambda x : serch_korean(x))].reset_index(drop = True)

In [23]:
len(non_unrated_problem_df) - len(korean_problem_df)

7792

In [24]:
len(problem_df) - len(korean_problem_df)

16224

In [25]:
len(korean_problem_df)

6866

In [26]:
score_df['key'] = score_df['user_name'] + '-' + score_df['target']


in_problem_df_target_score_df = score_df.set_index('target').loc[korean_problem_df['problem_id'].tolist(), :].reset_index(drop = False).sort_values(['key', 'date']).reset_index(drop = True)

In [27]:
len(score_df) - len(in_problem_df_target_score_df)

2492363

In [28]:
in_problem_df_target_score_df['target'].nunique() == len(korean_problem_df)

True

In [29]:
in_problem_df_problem_id_score_df = in_problem_df_target_score_df.set_index('problem_id').loc[korean_problem_df['problem_id'].tolist(), :].reset_index(drop = False).sort_values(['key', 'date']).reset_index(drop = True)

In [30]:
len(in_problem_df_target_score_df) - len(in_problem_df_problem_id_score_df)

264882

In [31]:
len(score_df) - len(in_problem_df_problem_id_score_df)

2757245

In [33]:
in_problem_df_target_score_df

,target,score_id,user_name,problem_id,answer,memory,time,language,length,date,key
0,11312,41729201,0,13548,맞았습니다!!,6032.0,280.0,C++17,1060.0,1649476228,0-11312
1,11312,41745912,0,12999,맞았습니다!!,6420.0,312.0,C++17,1083.0,1649501981,0-11312
2,11312,41746014,0,12986,맞았습니다!!,6420.0,192.0,C++17,1083.0,1649502114,0-11312
3,11312,41764127,0,15576,맞았습니다!!,44124.0,436.0,C++17,1583.0,1649552380,0-11312
4,11312,41816779,0,2783,맞았습니다!!,2020.0,0.0,C++17,284.0,1649660399,0-11312
...,...,...,...,...,...,...,...,...,...,...,...
2249855,23307,34881902,한의엽,20124,맞았습니다!!,7904.0,60.0,C++17,397.0,1635513231,한의엽-23307
2249856,23307,34882197,한의엽,20127,맞았습니다!!,2256.0,12.0,C++17,1015.0,1635513725,한의엽-23307
2249857,23307,34882210,한의엽,20929,맞았습니다!!,29432.0,228.0,C++17,1364.0,1635513759,한의엽-23307
2249858,17494,15381657,황산해,17492,100점,1116.0,0.0,C++14,748.0,1569652849,황산해-17494


In [32]:
len(in_problem_df_problem_id_score_df)

1984978

In [22]:
in_problem_df_problem_id_score_df['problem_id'].nunique() == len(korean_problem_df)

True

In [23]:
korean_problem_df.to_csv(os.path.join(DATA_PATH, 'clean_problem.csv'), index = False)
in_problem_df_problem_id_score_df.to_csv(os.path.join(DATA_PATH, 'clean_score.csv'), index = False)